# Enunciado

Se desechan $150$ kg/s de agua residual $(R_{f}=6\times10^{-5}~\mathrm{m}^2~\mathrm{K}/W)$ a $90~^\circ$C luego de pasar por el sistema de alcantarillado de la ciudad. Para disminuir la temperatura de salida, se opta por utilizar un intercambiador de placas. Cada placa del intercambiador tiene una distancia vertical entre puertos de $1.60$ metros y un ancho de $50$ centímetros, con una separación de $6$ milímetros entre placas. Según el fabricante, el rendimiento térmico se ve mejorado por un factor de amplaición superficial de $1.17$, y las placas Chevrón disponen de un ángulo de $50~^\circ$. Las placas son de Cuproníquel 90/10 y tienen un espesor de $0.8$ milímetros. El diámetro de los puertos está especificado como $0.15$ metros.

Con tal de reducir la temperatura del agua residual hasta los $60~^\circ$C, se busca utilizar agua de río que se encuentra naturalmente a $15~^\circ$C con un caudal másico de $200$ kg/s. Considere un arreglo de paso único para ambas corrientes (fría y caliente) a contra-corriente y un factor de seguridad de $1.5$.

1. Encuentre el número de platos requeridos (y totales) en el intercambiador, y determine el área de transferencia de calor efectiva para dicho número de platos.
2. A partir de sus resultados en el inciso anterior, calcule las péridas de carga totales para el fluido caliente y frio.

# Solución

In [1]:
Ti_h = 90 + 273.15
To_h = 60 + 273.15
Ti_c = 15 +273.15
m_h = 150
m_c = 200
Rf_c = 0.000086
Rf_h = 0.00006
k_w = 35
L_v = 1.6
L_w = 0.5
b = 6e-3
phi = 1.17
t = 8e-4
D_p = 0.15
C_s = 1.5
F = 1 #Supuesto

In [2]:
# Propiedades termofísicas, fluido frio
rho_c = 996
mu_c = 8.15e-4
cp_c = 4179
Pr_c = 5.58
k_c = 0.612
# Propiedades termofísicas, fluido caliente
rho_h = 965
mu_h = 3.16e-4
cp_h = 4205
Pr_h = 1.96
k_h = 0.675

## Inciso 1

In [3]:
def calc_h(N_t, m, b, L_w, D_e, mu, Pr, k):
    N_p = 1
    N_cp = (N_t - 1)/(2*N_p)
    mch = m/N_cp
    Gch = mch/(b*L_w)
    Re = Gch*D_e/mu
    Nu = 0.13*Re**0.732*Pr**(1/3)
    return Nu*k/D_e

### Encontrar $q$ y $\Delta T_{ml}$

In [4]:
import numpy as np
Q_r = m_h*cp_h*(Ti_h - To_h)
To_c = Q_r/(m_c*cp_c) + Ti_c
DTml = ( (To_h-Ti_c) - (Ti_h-To_c) )/np.log( (To_h-Ti_c)/(Ti_h-To_c) )
print(f'Q_r = {Q_r/1000:0.3f} kW')
print(f'DTml = {DTml:0.2f} °C')

Q_r = 18922.500 kW
DTml = 48.59 °C


### Número de placas a partir de un $U_f$ de diseño

In [5]:
U_f = 5000
Q_f = C_s * Q_r
A_e = Q_f/(U_f*F*DTml)
A_1p = (L_v - D_p) * L_w
A_1 = A_1p * phi
N_e = np.ceil(A_e/A_1) + 1 # Debe ser impar
print(f' Número de placas efectivo de {A_e/A_1:0.2f} redondeado a {N_e}')
N_t = N_e + 2
print(f'A_e = {A_e:0.3f} m2')
print(f'N_t = {N_t:0.0f}')

 Número de placas efectivo de 137.74 redondeado a 139.0
A_e = 116.836 m2
N_t = 141


### Verificar

In [6]:
A_ch = b*L_w
D_e = 2*b/phi
N_p = 1
h_h = calc_h(N_t, m_h, b, L_w, D_e, mu_h, Pr_h, k_h)
h_c = calc_h(N_t, m_c, b, L_w, D_e, mu_c, Pr_c, k_c)
U_c = 1/( 1/h_c + 1/h_h + t/k_w )
U_f = 1/( 1/U_c + Rf_c + Rf_h )
print(f'h_h = {h_h:0.2f} W/m2K')
print(f'h_c = {h_c:0.2f} W/m2K')
print(f'U_f = {U_f:0.2f} W/m2K')

h_h = 16787.36 W/m2K
h_c = 13309.08 W/m2K
U_f = 3294.22 W/m2K


Ya que 5000 $\neq$ 3294.22, se itera.

### Replicar con nuevo $U_f$

In [7]:
A_e = Q_f/(U_f*F*DTml)
N_e = np.ceil(A_e/A_1) + 1 # Debe ser impar
print(f' Número de placas efectivo de {A_e/A_1:0.2f} redondeado a {N_e}')
N_t = N_e + 2
print(f'A_e = {A_e:0.3f} m2')
print(f'N_t = {N_t:0.0f}')

 Número de placas efectivo de 209.06 redondeado a 211.0
A_e = 177.336 m2
N_t = 213


### Verificar

In [8]:
h_h = calc_h(N_t, m_h, b, L_w, D_e, mu_h, Pr_h, k_h)
h_c = calc_h(N_t, m_c, b, L_w, D_e, mu_c, Pr_c, k_c)
U_c = 1/( 1/h_c + 1/h_h + t/k_w )
U_f = 1/( 1/U_c + Rf_c + Rf_h )
print(f'h_h = {h_h:0.2f} W/m2K')
print(f'h_c = {h_c:0.2f} W/m2K')
print(f'U_f = {U_f:0.2f} W/m2K')

h_h = 12389.97 W/m2K
h_c = 9822.82 W/m2K
U_f = 2845.99 W/m2K


Ya que 3294.22 $\neq$ 2845.99, se itera.

### Replicar con nuevo $U_f$

In [9]:
A_e = Q_f/(U_f*F*DTml)
N_e = np.ceil(A_e/A_1) # Debe ser impar
print(f' Número de placas efectivo de {A_e/A_1:0.2f} redondeado a {N_e}')
N_t = N_e + 2
print(f'A_e = {A_e:0.3f} m2')
print(f'N_t = {N_t:0.0f}') # Notar que si N_t constante significa que U no cambiará. De todas formas se puede iterar para verificar.

 Número de placas efectivo de 241.99 redondeado a 242.0
A_e = 205.265 m2
N_t = 244


### Verificar

In [10]:
h_h = calc_h(N_t, m_h, b, L_w, D_e, mu_h, Pr_h, k_h)
h_c = calc_h(N_t, m_c, b, L_w, D_e, mu_c, Pr_c, k_c)
U_c = 1/( 1/h_c + 1/h_h + t/k_w )
U_f = 1/( 1/U_c + Rf_c + Rf_h )
print(f'h_h = {h_h:0.2f} W/m2K')
print(f'h_c = {h_c:0.2f} W/m2K')
print(f'U_f = {U_f:0.2f} W/m2K')

h_h = 11212.03 W/m2K
h_c = 8888.94 W/m2K
U_f = 2698.72 W/m2K


Ya que 2845.99 $\neq$ 2698.72, se itera.

### Replicar con nuevo $U_f$

In [11]:
A_e = Q_f/(U_f*F*DTml)
N_e = np.ceil(A_e/A_1) + 1 # Debe ser impar
print(f' Número de placas efectivo de {A_e/A_1:0.2f} redondeado a {N_e}')
N_t = N_e + 2
print(f'A_e = {A_e:0.3f} m2')
print(f'N_t = {N_t:0.0f}') # Notar que si N_t constante significa que U no cambiará. De todas formas se puede iterar para verificar.

 Número de placas efectivo de 255.19 redondeado a 257.0
A_e = 216.467 m2
N_t = 259


### Verificar

In [12]:
h_h = calc_h(N_t, m_h, b, L_w, D_e, mu_h, Pr_h, k_h)
h_c = calc_h(N_t, m_c, b, L_w, D_e, mu_c, Pr_c, k_c)
U_c = 1/( 1/h_c + 1/h_h + t/k_w )
U_f = 1/( 1/U_c + Rf_c + Rf_h )
print(f'h_h = {h_h:0.2f} W/m2K')
print(f'h_c = {h_c:0.2f} W/m2K')
print(f'U_f = {U_f:0.2f} W/m2K')

h_h = 10731.06 W/m2K
h_c = 8507.62 W/m2K
U_f = 2634.45 W/m2K


Ya que 2698.72 $\approx$ 2634.45, se tiene la verdadera área efectiva final.

### Área final

In [13]:
A_e = N_e*A_1
print(f'La verdadera área efectiva del intercambiador para {N_e:0.0f} placas efectivas será de {A_e:0.2f} m2')

La verdadera área efectiva del intercambiador para 257 placas efectivas será de 218.00 m2


## Inciso 2

In [14]:
def calc_Dpc(N_t, m, b, L_w, D_e, mu, rho):
    N_p = 1
    N_cp = (N_t - 1)/(2*N_p)
    mch = m/N_cp
    Gch = mch/(b*L_w)
    Re = Gch*D_e/mu
    f = 0.772/Re**0.161
    return 4*f * L_v*N_p/D_e * Gch**2/(2*rho)

In [15]:
# Canales
Dpc_h = calc_Dpc(N_t, m_h, b, L_w, D_e, mu_h, rho_h)
Dpc_c = calc_Dpc(N_t, m_c, b, L_w, D_e, mu_c, rho_c)
# Puertos
Gp_h = m_h/(np.pi/4 * D_p**2)
Gp_c = m_c/(np.pi/4 * D_p**2)
Dpp_h = 1.4*1 * Gp_h**2/(2*rho_h)
Dpp_c = 1.4*1 * Gp_c**2/(2*rho_c)
Dp_h = Dpc_h + Dpp_h
Dp_c = Dpc_c + Dpp_c
print(f'Caída de presión del fluido caliente: {Dp_h/1000:0.2f} kPa')
print(f'Caída de presión del fluido frío: {Dp_c/1000:0.2f} kPa')

Caída de presión del fluido caliente: 60.47 kPa
Caída de presión del fluido frío: 105.74 kPa
